In [4]:
# ============================================
# DEMOCRACIA VERDADERA - Optimización con AG
# ============================================

import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms

# Reproducibilidad
random.seed(1)
np.random.seed(1)

# -------------------------------
# 1) Parámetros iniciales
# -------------------------------
n_partidos = 5
n_curules = 50
n_entidades = 50

# -------------------------------
# 2) Distribución no uniforme de curules
# -------------------------------
curules = np.random.multinomial(n_curules, np.random.dirichlet(np.ones(n_partidos)*2))
df_curules = pd.DataFrame({
    "Partido": [f"Partido {i+1}" for i in range(n_partidos)],
    "Curules": curules
})
total_curules = df_curules["Curules"].sum()
proporcion_curules = df_curules["Curules"] / total_curules

print("Distribución inicial de curules (poder en el Congreso):")
print(df_curules.to_string(index=False))

# -------------------------------
# 3) Definición de entidades con pesos
# -------------------------------
entidades = [f"Entidad {i+1}" for i in range(n_entidades)]
pesos = np.random.randint(1, 101, size=n_entidades)

df_entidades = pd.DataFrame({
    "Entidad": entidades,
    "Peso": pesos
})

print("\nLista de entidades y sus pesos:")
pd.set_option('display.max_rows', None)
print(df_entidades.to_string(index=False))
pd.reset_option('display.max_rows')

# -------------------------------
# 4) Algoritmo Genético (DEAP)
# -------------------------------

# Fitness: minimizar diferencia cuadrática entre proporciones
for name in ("FitnessMin", "Individual"):
    if hasattr(creator, name):
        delattr(creator, name)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Un individuo: lista de 50 enteros (partido asignado a cada entidad)
def create_individual():
    return [random.randint(0, n_partidos - 1) for _ in range(n_entidades)]

toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluar(individuo):
    # poder asignado por partido
    poder_asignado = [0] * n_partidos
    for i, partido in enumerate(individuo):
        poder_asignado[partido] += pesos[i]

    total_poder = sum(poder_asignado)
    proporciones_asignadas = [p / total_poder for p in poder_asignado]

    # Diferencia cuadrática respecto a proporciones de curules
    diff = sum((proporciones_asignadas[i] - proporcion_curules[i])**2 for i in range(n_partidos))
    return (diff,)

toolbox.register("evaluate", evaluar)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=n_partidos-1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# -------------------------------
# 5) Ejecución del AG
# -------------------------------
population = toolbox.population(n=200)
hof = tools.HallOfFame(1)

algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=200, halloffame=hof, verbose=False)

best = hof[0]

# -------------------------------
# 6) Resultados finales
# -------------------------------
# Calcular poder final asignado
poder_final = [0] * n_partidos
for i, partido in enumerate(best):
    poder_final[partido] += pesos[i]

df_resultado = pd.DataFrame({
    "Partido": [f"Partido {i+1}" for i in range(n_partidos)],
    "Curules": df_curules["Curules"],
    "Porc_Curules": proporcion_curules * 100,
    "Poder_Entidades": poder_final,
    "Porc_Poder": (np.array(poder_final) / sum(poder_final)) * 100
})

print("\n=== Comparación final de poder y curules ===")
print(df_resultado.to_string(index=False))


Distribución inicial de curules (poder en el Congreso):
  Partido  Curules
Partido 1       23
Partido 2        1
Partido 3        5
Partido 4        2
Partido 5       19

Lista de entidades y sus pesos:
   Entidad  Peso
 Entidad 1    95
 Entidad 2    97
 Entidad 3    87
 Entidad 4    14
 Entidad 5    10
 Entidad 6     8
 Entidad 7    64
 Entidad 8    62
 Entidad 9    23
Entidad 10    58
Entidad 11     2
Entidad 12     1
Entidad 13    61
Entidad 14    82
Entidad 15     9
Entidad 16    89
Entidad 17    14
Entidad 18    48
Entidad 19    73
Entidad 20    31
Entidad 21    72
Entidad 22     4
Entidad 23    71
Entidad 24    22
Entidad 25    50
Entidad 26    58
Entidad 27     4
Entidad 28    69
Entidad 29    25
Entidad 30    44
Entidad 31    77
Entidad 32    27
Entidad 33    53
Entidad 34    81
Entidad 35    42
Entidad 36    83
Entidad 37    16
Entidad 38    65
Entidad 39    69
Entidad 40    26
Entidad 41    99
Entidad 42    88
Entidad 43     8
Entidad 44    27
Entidad 45    26
Entidad 46    2

Se puede apreciar una cercanía entre las proporciones del número de curules y el poder asignado en entidades.
Por ejemplo, para el partido 1 cuya proporción de curules es
23/50 =0,46(46%), el poder en entidades es 1049/2287≈0,45822871049≈0,458, es decir, también cerca del 46%.